## AnimateDiff

### 1. Env Setting

In [ ]:
conda activate anima
cd AnimateDiff

### 2. Select pre-trained cofig

In [ ]:
#original config
configs/prompts/1_animate/1_1_animate_RealisticVision.yaml
configs/prompts/1_animate/1_2_animate_FilmVelvia.yaml
configs/prompts/1_animate/1_3_animate_ToonYou.yaml
configs/prompts/1_animate/1_4_animate_MajicMix.yaml
configs/prompts/1_animate/1_5_animate_RcnzCartoon.yaml
configs/prompts/1_animate/1_6_animate_Lyriel.yaml
configs/prompts/1_animate/1_7_animate_Tusun.yaml

### 3. Launch in Linux

In [ ]:
CUDA_VISIBLE_DEVICES=2 python -m scripts.animate \
    --config configs/prompts/1_animate/1_3_animate_ToonYou.yaml \
    --my_prompt '/userHome/userhome4/demo/baby_video/Baby_Video/inference_code/240919_prompts.txt' \
    --save_dir '/userHome/userhome4/demo/baby_video/Baby_Video/inference_outputs/animatediff'

In [ ]:
CUDA_VISIBLE_DEVICES=0 python -m scripts.animate \
    --config configs/prompts/1_animate/1_4_animate_MajicMix.yaml \
    --my_prompt '/userHome/userhome4/demo/baby_video/Baby_Video/inference_code/240919_prompts.txt' \
    --save_dir '/userHome/userhome4/demo/baby_video/Baby_Video/inference_outputs/animatediff'

In [ ]:
CUDA_VISIBLE_DEVICES=2 python -m scripts.animate \
    --config configs/prompts/1_animate/1_5_animate_RcnzCartoon.yaml \
    --my_prompt '/userHome/userhome4/demo/baby_video/Baby_Video/inference_code/240919_prompts.txt' \
    --save_dir '/userHome/userhome4/demo/baby_video/Baby_Video/inference_outputs/animatediff'

## Free-Bloom

### 1. Env Setting

In [ ]:
conda activate fb
cd Free-Bloom

### 2. Launch in Linux

In [ ]:
python main.py \
    --config configs/flowers.yaml \
    --my_prompt '/userHome/userhome4/demo/baby_video/Baby_Video/inference_code/240919_prompts.txt' \
    --save_dir '/userHome/userhome4/demo/baby_video/Baby_Video/inference_outputs/free-bloom'

## CogVideoX


### (실행안됨 - 다시 테스트 필요)
### 1. Launch by HuggingFace

In [ ]:
#env setting: conda activate vgen

import datetime, os
import torch
from diffusers import CogVideoXPipeline
from diffusers.utils import export_to_video

my_prompt='/userHome/userhome4/demo/baby_video/Baby_Video/inference_code/240919_prompts.txt'
time_str = datetime.datetime.now().strftime("%Y-%m-%d")
save_dir=os.path.join('/userHome/userhome4/demo/baby_video/Baby_Video/inference_outputs/free-bloom', time_str)
os.makedirs(save_dir, exist_ok=True)

with open(my_prompt, "r") as f:
    prompts = [line.strip() for line in f.readlines()]

for idx, prompt in enumerate(prompts):
    
    pipe = CogVideoXPipeline.from_pretrained(
        "THUDM/CogVideoX-2b",
        torch_dtype=torch.float16
    )

    pipe.enable_model_cpu_offload()
    pipe.enable_sequential_cpu_offload()
    pipe.vae.enable_slicing()
    pipe.vae.enable_tiling()
    video = pipe(
        prompt=prompt,
        num_videos_per_prompt=1,
        num_inference_steps=50,
        num_frames=49,
        guidance_scale=6,
        generator=torch.Generator(device="cuda").manual_seed(42),
    ).frames[0]

    export_to_video(video, f"{save_dir}/{idx}_prompt.mp4", fps=8)